In [1]:
!pip install tensorflow pandas numpy
!pip install kaggle  


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.7 kB ? eta -:--:--
     ------------------ ------------------- 41.0/82.7 kB 164.3 kB/s eta 0:00:01
     -------------------------------------  81.9/82.7 kB 328.8 kB/s eta 0:00:01
     -------------------------------------- 82.7/82.7 kB 257.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.2 kB 2.0 MB/s eta 0:00:01
   ------------------------------------ --- 71.7/78.2 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 78.2/78.2 kB 730.3 kB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pandas numpy tensorflow keras


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
# Load English-French dataset
df_french = pd.read_csv('eng_-french.csv') 
df_french.columns = ['English', 'French']  

# Load English-Hindi dataset
df_hindi = pd.read_csv('Dataset_English_Hindi.csv')  
df_hindi.columns = ['English', 'Hindi']  

In [7]:
# Combine datasets
df_combined = pd.merge(df_french, df_hindi, on='English', how='outer')

# Drop rows with any NaN values
df_combined.dropna(inplace=True)

# Shuffle the dataset
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

# Check the combined dataset
df_combined.head()

,English,French,Hindi
0,She told me that she wanted a pet dog.,Elle me dit qu'elle voulait un chien de compag...,उसने मुझसे कहा कि उसे कुत्ता पालने का मन था।
1,Thank you very much.,Merci bien.,बहुत बहुत धन्यवाद.
2,What do you think she is doing now?,Que pensez-vous qu'elle soit maintenant en tra...,तुम्हारे हिसाब से वह अभी क्या कर रही होगी?
3,Have you been abroad?,Avez-vous déjà été à l'étranger ?,आप विदेश गए हुए हैं क्या?
4,"Who is it? ""It's me.""",« Qui est là ? » « C'est moi. »,"कौन है? ""मैं हूँ"""


In [8]:
# Split the data into training and testing sets
X = df_combined['English']
y_french = df_combined['French']
y_hindi = df_combined['Hindi']

X_train, X_test, y_train_french, y_test_french, y_train_hindi, y_test_hindi = train_test_split(
    X, y_french, y_hindi, test_size=0.2, random_state=42)

# Check the shapes of the data
print(X_train.shape, X_test.shape, y_train_french.shape, y_test_french.shape, y_train_hindi.shape, y_test_hindi.shape)

(2348,) (588,) (2348,) (588,) (2348,) (588,)


In [9]:
# Tokenization
tokenizer_en = tf.keras.preprocessing.text.Tokenizer()
tokenizer_en.fit_on_texts(X_train)

tokenizer_fr = tf.keras.preprocessing.text.Tokenizer()
tokenizer_fr.fit_on_texts(y_train_french)

tokenizer_hi = tf.keras.preprocessing.text.Tokenizer()
tokenizer_hi.fit_on_texts(y_train_hindi)

# Convert texts to sequences
X_train_seq = tokenizer_en.texts_to_sequences(X_train)
y_train_fr_seq = tokenizer_fr.texts_to_sequences(y_train_french)
y_train_hi_seq = tokenizer_hi.texts_to_sequences(y_train_hindi)

X_test_seq = tokenizer_en.texts_to_sequences(X_test)
y_test_fr_seq = tokenizer_fr.texts_to_sequences(y_test_french)
y_test_hi_seq = tokenizer_hi.texts_to_sequences(y_test_hindi)

# Pad sequences
max_length = max([len(seq) for seq in X_train_seq])
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Check shapes of padded sequences
print(X_train_pad.shape, X_test_pad.shape)

(2348, 16) (588, 16)


In [11]:
# Model parameters
vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1
vocab_size_hi = len(tokenizer_hi.word_index) + 1
embedding_dim = 128

# Build the model
def create_model(vocab_size, max_length):
    model = keras.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(128, activation='relu'),
        layers.Dense(vocab_size, activation='softmax')
    ])
    return model

# Create models for French and Hindi translations
model_fr = create_model(vocab_size_fr, max_length)
model_hi = create_model(vocab_size_hi, max_length)

# Compile the models
model_fr.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_hi.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# Prepare y for training (only use the first token of the target sequence)
y_train_fr = np.array([seq[0] for seq in y_train_fr_seq])
y_train_hi = np.array([seq[0] for seq in y_train_hi_seq])

# Train the French translation model
model_fr.fit(X_train_pad, y_train_fr, epochs=10, validation_split=0.2)

# Train the Hindi translation model
model_hi.fit(X_train_pad, y_train_hi, epochs=10, validation_split=0.2)

Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.1063 - loss: 6.5458 - val_accuracy: 0.1170 - val_loss: 4.7863
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1336 - loss: 4.4046 - val_accuracy: 0.1000 - val_loss: 4.7707
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1152 - loss: 4.2799 - val_accuracy: 0.1170 - val_loss: 4.8033
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.1258 - loss: 4.3356 - val_accuracy: 0.1170 - val_loss: 4.8452
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1380 - loss: 4.2508 - val_accuracy: 0.1170 - val_loss: 4.8537
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1302 - loss: 4.3407 - val_accuracy: 0.1170 - val_loss: 4.8522
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1124 - loss: 4.3022 - val_accuracy: 0.1170 - val_loss: 4.8878
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1340 - loss: 4.2848 - val_accuracy: 0.1170 - 

In [14]:
# Save the trained models
model_fr.save('english_to_french_model.keras')
model_hi.save('english_to_hindi_model.keras')

In [18]:
# Convert the target sequences to numpy arrays for evaluation
y_test_fr_array = np.array([seq[0] if len(seq) > 0 else 0 for seq in y_test_fr_seq])
y_test_hi_array = np.array([seq[0] if len(seq) > 0 else 0 for seq in y_test_hi_seq])

# Evaluate the models
fr_loss, fr_acc = model_fr.evaluate(X_test_pad, y_test_fr_array, verbose=0)
hi_loss, hi_acc = model_hi.evaluate(X_test_pad, y_test_hi_array, verbose=0)

print(f'French Translation Model Accuracy: {fr_acc}')
print(f'Hindi Translation Model Accuracy: {hi_acc}')

French Translation Model Accuracy: 0.11224489659070969
Hindi Translation Model Accuracy: 0.15986394882202148


In [19]:
print(f'X_test_pad shape: {X_test_pad.shape}')
print(f'y_test_fr_array shape: {y_test_fr_array.shape}')
print(f'y_test_hi_array shape: {y_test_hi_array.shape}')

X_test_pad shape: (588, 16)
y_test_fr_array shape: (588,)
y_test_hi_array shape: (588,)


In [20]:
def translate(text):
    # Tokenize and pad the input
    input_seq = tokenizer_en.texts_to_sequences([text])
    input_pad = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_length, padding='post')

    # Predict French
    fr_pred = model_fr.predict(input_pad)
    fr_word_index = np.argmax(fr_pred, axis=-1)
    fr_word = tokenizer_fr.index_word[fr_word_index[0]]

    # Predict Hindi
    hi_pred = model_hi.predict(input_pad)
    hi_word_index = np.argmax(hi_pred, axis=-1)
    hi_word = tokenizer_hi.index_word[hi_word_index[0]]

    return fr_word, hi_word

# Example usage
english_word = "example"
french_translation, hindi_translation = translate(english_word)
print(f"French: {french_translation}, Hindi: {hindi_translation}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
French: merci, Hindi: तुम
